In [19]:
#RNN 예시

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

In [13]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.rnn = nn.RNN(input_size, hidden_size)   # RNN 레이어
        self.fc = nn.Linear(hidden_size, output_size)   # 출력 레이어

    def forward(self, x):
        # 초기 은닉 상태 설정
        h0 = torch.zeros(1, x.size(1), self.hidden_size)
        
        # RNN에 x와 초기 은닉 상태 전달
        out, _ = self.rnn(x, h0)  # x에 배치 차원 추가
        
        # 마지막 단계의 출력을 분류기에 전달
        out = self.fc(out[:, -1, :]) 
        return out

In [17]:
# 입력 및 출력 사이즈 정의
input_size = 10 # 입력 특성의 수
hidden_size = 20 # 은닉 상태의 특성 수
output_size = 1 # 출력 특성의 수

# 임의의 훈련 데이터 생성
x_train = torch.randn(100, 5, input_size)  # 100개의 샘플, 각 샘플은 5시간 단계, 10개의 특성을 가짐
y_train = torch.randn(100, 1)  # 각 샘플에 대한 임의의 출력값

# 테스트 데이터
x_test = torch.randn(20, 5, input_size)  
y_test = torch.randn(20, 1)

In [15]:
# 모델 초기화
model = SimpleRNN(input_size, hidden_size, output_size)

criterion = nn.MSELoss()  #손실함수
optimizer = optim.Adam(model.parameters(), lr=0.01) #옵티마이저:계산된 기울기를 기반으로 매개변수를 업데이트

In [16]:
epochs = 10  # 훈련 에폭 수

for epoch in range(epochs):
    for i in range(len(x_train)):
        # 현재 배치 데이터
        inputs = x_train[i].unsqueeze(1)  # 배치 차원 추가
        labels = y_train[i]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

e:\anaconda3\envs\llm\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 0.9910
Epoch [2/10], Loss: 1.7067
Epoch [3/10], Loss: 1.8483
Epoch [4/10], Loss: 1.8315
Epoch [5/10], Loss: 2.7526
Epoch [6/10], Loss: 2.3211
Epoch [7/10], Loss: 1.3417
Epoch [8/10], Loss: 1.4228
Epoch [9/10], Loss: 1.2000
Epoch [10/10], Loss: 0.9052


In [18]:
# 모델을 평가 모드로 설정
model.eval()

test_loss = 0
with torch.no_grad():
    for i in range(len(x_test)):
        inputs = x_test[i].unsqueeze(1)
        labels = y_test[i]

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss / len(x_test):.4f}')


Test Loss: 0.9136
